## Gene-related condition/disorders

* Pull out all traits associated with this submission/submitter (?)
    * Needs to be by [submitter ID](https://www.ncbi.nlm.nih.gov/clinvar/submitters/239772/)
    * Shouldn't use this as a long-term solution, just to ensure we get the right "ground truth" set
* Does regex like `[0-9a-zA-Z]+-related .*` work?
    * Precision/recall over this set
* How many such traits have EFO/MONDO/HP terms?
* How many such traits have Medgen terms?
* How many such traits have records from other submitters?
* How many associated variants have other records?
    * Note we already know 99% of targets are covered by other records
* Iterate on the regex as needed

In [11]:
import sys
sys.path.append('..')

In [12]:
from filter_clinvar_xml import filter_xml, pprint, iterate_cvs_from_xml

from cmat.clinvar_xml_io import *
from cmat.clinvar_xml_io.xml_parsing import *

import gzip
import os
import re
import pandas as pd

/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/output_generation/clinvar_to_evidence_strings.py:198: SyntaxWarning: invalid escape sequence '\d'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/output_generation/clinvar_to_evidence_strings.py:198: SyntaxWarning: invalid escape sequence '\d'


In [4]:
data_dir = os.getenv('WORK_DIR')
full_clinvar_xml = os.path.join(data_dir, 'full-clinvar.xml.gz')
prevention_xml = os.path.join(data_dir, 'prevention-records.xml.gz')

In [5]:
GENE_RELATED_DISORDER = r'^\S+-related disorder$'

In [7]:
prevention_id = '239772'

In [ ]:
header = b'''<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<ReleaseSet Dated="." xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Type="full" xsi:noNamespaceSchemaLocation="http://ftp.ncbi.nlm.nih.gov/pub/clinvar/xsd_public/clinvar_public_2.0.xsd">
'''
count = 0
with gzip.open(prevention_xml, 'wb') as output_file:
    output_file.write(header)
    for raw_cvs_xml in iterate_cvs_from_xml(full_clinvar_xml):
        
        # 1. Trait must have a valid name
        rcv = find_mandatory_unique_element(raw_cvs_xml, 'ReferenceClinVarAssertion')
        record = ClinVarRecord(rcv, 2.0)
        if len(record.traits_with_valid_names) == 0:
            continue
        
        # 2. Record must have a PreventionGenetics submission
        subs = find_elements(raw_cvs_xml, 'ClinVarAssertion/ClinVarAccession')
        for s in subs:
            org_id = s.attrib['OrgID']
            if str(org_id) == prevention_id:
                output_file.write(ElementTree.tostring(raw_cvs_xml))
                count += 1
                
    output_file.write(b'</ReleaseSet>')
print(f'Records written: {count}')

# Records written: 112101

In [9]:
prevention_dataset = ClinVarDataset(prevention_xml)

In [10]:
prevention_trait_names = set()
for r in prevention_dataset:
    for t in r.traits_with_valid_names:
        prevention_trait_names.add(t.preferred_or_other_valid_name)

In [11]:
len(prevention_trait_names)

5745

In [12]:
prevention_trait_names = list(prevention_trait_names)

In [28]:
not_related_disorder = [name for name in prevention_trait_names if not re.match(GENE_RELATED_DISORDER, name)]

In [35]:
len(not_related_disorder)

18

In [29]:
# Indicates we can't filter on submitter either
# Also note *-related condition is indeed removed entirely - nothing to stop anyone from using another phrase for something similarly generic
not_related_disorder

['Early-onset progressive diffuse brain atrophy-microcephaly-muscle weakness-optic atrophy syndrome',
 'Niemann-Pick disease, type A',
 'Congenital heart defects, dysmorphic facial features, and intellectual developmental disorder',
 'Sandhoff disease',
 'Spinocerebellar ataxia type 12',
 'Von Hippel-Lindau syndrome',
 'Polycystic kidney disease, adult type',
 'Tyrosinase-negative oculocutaneous albinism',
 'Mitral valve prolapse, myxomatous 2',
 'Gaucher disease type I',
 'Gaucher disease type III',
 'Distinctive facial features',
 'Multiple congenital anomalies',
 'Gaucher disease type II',
 'Van Maldergem syndrome 1',
 'Anterior segment dysgenesis 7',
 'Developmental delay',
 'ASAH1-related disorders']

Don't think this is the right set to be looking at, could also do submission name (SUB14299258) or date (2024-03-08) to target the specific problematic submission but this also doesn't really address the root cause.

This also raises the question as to whether we should filter gene-related condition as well as disorder, even though it's disappeared from this specific case it's still arguably not specific enough to be annotated properly. Maybe should leave it until it becomes a concrete problem though.

In [6]:
related_disorder_xml = os.path.join(data_dir, 'disorder-records.xml.gz')

In [33]:
# Take another set, this time any with preferred trait name ending with "related disorder"

def has_related_disorder_trait(x: ClinVarRecord):
    for t in x.traits_with_valid_names:
        if re.match(GENE_RELATED_DISORDER, t.preferred_or_other_valid_name):
            return True
    return False


filter_xml(
    input_xml=full_clinvar_xml,
    output_xml=related_disorder_xml,
    filter_fct=has_related_disorder_trait,
    max_num=None,
)

INFO:filter_clinvar_xml:Records written: 122432


In [7]:
disorder_dataset = ClinVarDataset(related_disorder_xml)

In [41]:
import logging

In [42]:
logging.getLogger("cmat.clinvar_xml_io.clinvar_trait").setLevel(logging.ERROR)

In [43]:
# Check coverage of MedGen terms within ClinVar
n_disorder_traits = 0
n_disorder_traits_with_medgen = 0
trait_to_medgen = {}

for record in disorder_dataset:
    for trait in record.traits_with_valid_names:
        if re.match(GENE_RELATED_DISORDER, trait.preferred_or_other_valid_name):
            n_disorder_traits += 1
            if trait.medgen_id:
                n_disorder_traits_with_medgen += 1
                trait_to_medgen[trait.preferred_or_other_valid_name] = trait.medgen_id

In [44]:
n_disorder_traits

122432

In [45]:
n_disorder_traits_with_medgen

12006

In [55]:
len(trait_to_medgen)

80

In [47]:
n_disorder_traits_with_medgen / n_disorder_traits

0.09806259801359121

12,006 / 122,432 = 9.8% of these traits have MedGen terms, not clear how meaningful they are. Two examples picked at random:
* [TCTN2-related disorder](https://www.ncbi.nlm.nih.gov/medgen/?term=TCTN2-related+disorder)
* [ZMPSTE24-related disorder](https://www.ncbi.nlm.nih.gov/medgen/?term=ZMPSTE24-related+disorder)


In [48]:
from cmat.trait_mapping.ols import get_uri_from_exact_match

In [49]:
logging.getLogger("cmat.trait_mapping").setLevel(logging.ERROR)

In [51]:
# Check coverage of EFO terms via OLS

n_disorder_traits_with_efo = 0
trait_to_efo = {}

for record in disorder_dataset:
    for trait in record.traits_with_valid_names:
        if re.match(GENE_RELATED_DISORDER, trait.preferred_or_other_valid_name):
            efo_uri = get_uri_from_exact_match(trait.preferred_or_other_valid_name.lower(), 'EFO')
            if efo_uri:
                n_disorder_traits_with_efo += 1
                trait_to_efo[trait.preferred_or_other_valid_name] = efo_uri

In [52]:
n_disorder_traits_with_efo

599

In [54]:
trait_to_efo

{'CBL-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0013308',
 'CLCN4-related disorder': 'http://www.ebi.ac.uk/efo/EFO_0009066',
 'ATP6AP2-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0100146',
 'STAG1-related disorder': 'http://www.ebi.ac.uk/efo/EFO_0009078',
 'COL4A1-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0800461',
 'DKC1-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0100152',
 'CTSC-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0800465'}

In [56]:
# Just get the complete set of names
unique_names = set()
for record in disorder_dataset:
    for trait in record.traits_with_valid_names:
        if re.match(GENE_RELATED_DISORDER, trait.preferred_or_other_valid_name):
            unique_names.add(trait.preferred_or_other_valid_name)

In [57]:
# Compare 5745-18=5727 within "prevention" set
len(unique_names)

5788

In [59]:
# Dump to file
with open(os.path.join(data_dir, 'disorder-traits.csv'), 'w+') as outfile:
    outfile.write('trait_name,medgen_id,efo_uri\n')
    for n in unique_names:
        outfile.write(f'{n},{trait_to_medgen.get(n, "")},{trait_to_efo.get(n, "")}\n')

In [60]:
len(trait_to_medgen) / len(unique_names)

0.013821700069108501

In [61]:
len(trait_to_efo) / len(unique_names)

0.0012093987560469939

In [63]:
# For a markdown table...
for t in trait_to_efo:
    print(f'{t}|{trait_to_efo[t]}')

CBL-related disorder|http://purl.obolibrary.org/obo/MONDO_0013308
CLCN4-related disorder|http://www.ebi.ac.uk/efo/EFO_0009066
ATP6AP2-related disorder|http://purl.obolibrary.org/obo/MONDO_0100146
STAG1-related disorder|http://www.ebi.ac.uk/efo/EFO_0009078
COL4A1-related disorder|http://purl.obolibrary.org/obo/MONDO_0800461
DKC1-related disorder|http://purl.obolibrary.org/obo/MONDO_0100152
CTSC-related disorder|http://purl.obolibrary.org/obo/MONDO_0800465


### Part 2

Check coverage of variants associated with gene-related disorder terms - are they covered under other terms as well?

In [9]:
# 1. Pull VCVs of measures in the disorder set
#    (Should we use another variant identifier, e.g. chr_pos_ref_alt?)
disorder_vcvs = set()

for record in disorder_dataset:
    if record.measure:
        disorder_vcvs.add(record.vcv)

In [11]:
len(disorder_vcvs)

122355

In [10]:
full_dataset = ClinVarDataset(full_clinvar_xml)

In [12]:
related_disorder_vcv_xml = os.path.join(data_dir, 'disorder-vcv-records.xml.gz')

In [13]:
from collections import defaultdict

In [31]:
# 2. Go through full set and pull terms/records associated with those VCVs
disorder_vcv_trait_names = defaultdict(set)


# NB. filter function with side effects!
def has_disorder_vcv(x: ClinVarRecord):
    if x.traits_with_valid_names and x.measure and x.vcv in disorder_vcvs:
        disorder_vcv_trait_names[x.vcv] |= {t.preferred_or_other_valid_name.lower() for t in x.traits_with_valid_names}
        return True
    return False


filter_xml(
    input_xml=full_clinvar_xml,
    output_xml=related_disorder_vcv_xml,
    filter_fct=has_disorder_vcv,
    max_num=None,
)


INFO:filter_clinvar_xml:Records written: 213830


In [32]:
len(disorder_vcv_trait_names)

122355

In [33]:
mult_traits = {
    k: v for k, v in disorder_vcv_trait_names.items() if len(v) > 1
}
sing_trait = {
    k: v for k, v in disorder_vcv_trait_names.items() if len(v) == 1
}

In [34]:
len(mult_traits)

49241

In [35]:
len(sing_trait)

73114

In [39]:
trait_to_efo = {'CBL-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0013308',
 'CLCN4-related disorder': 'http://www.ebi.ac.uk/efo/EFO_0009066',
 'ATP6AP2-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0100146',
 'STAG1-related disorder': 'http://www.ebi.ac.uk/efo/EFO_0009078',
 'COL4A1-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0800461',
 'DKC1-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0100152',
 'CTSC-related disorder': 'http://purl.obolibrary.org/obo/MONDO_0800465'}

In [40]:
traits_with_efo = {t.lower() for t in trait_to_efo}

In [53]:
n_mult_trait_vars_with_efo_trait = 0
n_sing_trait_vars_with_efo_trait = 0
to_output = {}

for vcv, traits in mult_traits.items():
    if len(traits & traits_with_efo) > 0:
        n_mult_trait_vars_with_efo_trait += 1
        to_output[vcv] = traits
        
for vcv, traits in sing_trait.items():
    if len(traits & traits_with_efo) > 0:
        n_sing_trait_vars_with_efo_trait += 1
        to_output[vcv] = traits

print('Total variants associated with gene-related disorder traits:', len(disorder_vcv_trait_names))
print('\tNumber of these with multiple traits:', len(mult_traits))
print('\t\tNumber of these including a currently EFO-mapped one:', n_mult_trait_vars_with_efo_trait)
print('\tNumber of these with single trait:', len(sing_trait))
print('\t\tNumber of these including a currently EFO-mapped one:', n_sing_trait_vars_with_efo_trait)

Total variants associated with gene-related disorder traits: 122355
	Number of these with multiple traits: 49241
		Number of these including a currently EFO-mapped one: 236
	Number of these with single trait: 73114
		Number of these including a currently EFO-mapped one: 350


In [49]:
49241/122355

0.40244370887989867

In [50]:
73114/122355

0.5975562911201013

In [54]:
# dump vcv-to-traits mapping (containing efo mapped traits only
with open(os.path.join(data_dir, 'vcv-to-efo-traits.tsv'), 'w+') as outfile:
    outfile.write('vcv\ttrait_names\n')
    for vcv, traits in to_output.items():
        outfile.write(f'{vcv}\t{",".join(traits)}\n')

## Revisiting in 2025

Further investigation when terms reappeared in January 2025.

In [1]:
import sys
sys.path.append('..')

In [2]:
from filter_clinvar_xml import pprint

/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/clinvar_xml_io/clinvar_record.py:54: SyntaxWarning: invalid escape sequence '\?'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/clinvar_xml_io/clinvar_record.py:54: SyntaxWarning: invalid escape sequence '\?'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/clinvar_xml_io/hgvs_variant.py:35: SyntaxWarning: invalid escape sequence '\.'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/clinvar_xml_io/hgvs_variant.py:35: SyntaxWarning: invalid escape sequence '\.'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg/cmat/output_generation/clinvar_to_evidence_strings.py:198: SyntaxWarning: invalid escape sequence '\d'
/home/ashen/projects/eva-opentargets/venv/lib/python3.12/site-packages/cmat-3.3.2-py3.12.egg

In [3]:
from cmat.clinvar_xml_io import *
from cmat.clinvar_xml_io.filtering import submission_names_to_exclude
from collections import Counter

In [4]:
previous_set = ClinVarDataset('/home/ashen/projects/opentargets/gene-condition/disorder-records.xml.gz')
current_set = ClinVarDataset('/home/ashen/projects/opentargets/gene-condition/2025_disorder-records.xml.gz')

In [29]:
def print_sub_counts(dataset):
    only_excluded_subs = 0
    excluded_and_others = 0
    only_other_subs = 0
    submission_names = Counter()
    
    for cvs in dataset.iter_cvs():
        has_excluded = False
        has_others = False
        for scv in cvs.scvs:
            submission_names[scv.submission_name] += 1
            if scv.submission_name in submission_names_to_exclude:
                has_excluded = True
            if scv.submission_name not in submission_names_to_exclude:
                has_others = True
        if has_excluded and not has_others:
            only_excluded_subs += 1
        if has_excluded and has_others:
            excluded_and_others += 1
        if not has_excluded and has_others:
            only_other_subs += 1

    print('Only supported by excluded submission:', only_excluded_subs)
    print('Supported by excluded and other submissions:', excluded_and_others)
    print('Only supported by other submissions:', only_other_subs)
    return submission_names

In [31]:
previous_sub_names = print_sub_counts(previous_set)

Only supported by excluded submission: 92629
Supported by excluded and other submissions: 351
Only supported by other submissions: 29452


In [32]:
current_sub_names = print_sub_counts(current_set)

Only supported by excluded submission: 428
Supported by excluded and other submissions: 0
Only supported by other submissions: 154135


In [33]:
submission_names_to_exclude

['SUB14299258']

In [3]:
excluded_submission = submission_names_to_exclude[0]

In [34]:
previous_sub_names

Counter({'SUB14299258': 92980,
         'SUB13915440': 17094,
         None: 6752,
         'SUB6641900': 950,
         'SUB10993736': 664,
         'SUB14200314': 642,
         'SUB14200283': 596,
         'SUB14200279': 436,
         'SUB14200269': 411,
         'SUB5118208': 407,
         'SUB13808474': 224,
         'ICSL_2016Q4': 168,
         'SUB12508014': 109,
         'SUB6300041': 101,
         'SUB11447241': 84,
         'SUB11035566': 60,
         'SUB12507860': 55,
         'GenomeConnect_hg19_81817': 53,
         'SUB12390657': 47,
         'SUB7304360': 41,
         'SUB14120880': 36,
         'SUB12506961': 35,
         'SUB5106164': 34,
         'SUB9483450': 31,
         'SUB8730666': 25,
         'Natera0001': 25,
         'SUB14430613': 24,
         'SUB14011620': 23,
         'SUB14505471': 20,
         'NGS_2019-2': 19,
         'SUB10524990': 19,
         'SUB3781225': 18,
         'NIHR_Bioresource_Rare_Diseases_MYH9': 18,
         'SUB13685737': 17,
         'S

In [35]:
current_sub_names

Counter({None: 101132,
         'SUB13915440': 15410,
         'SUB14767636': 5591,
         'SUB14767635': 3362,
         'SUB14767656': 2997,
         'SUB14767649': 2807,
         'SUB14767648': 2765,
         'SUB14767647': 2520,
         'SUB14767644': 2386,
         'SUB14767646': 2203,
         'SUB14767645': 2123,
         'SUB14767639': 1741,
         'SUB14767642': 1582,
         'SUB14767634': 1219,
         'SUB6641900': 950,
         'SUB14767640': 776,
         'SUB10993736': 664,
         'SUB14200314': 642,
         'SUB14200283': 596,
         'SUB14200279': 436,
         'SUB14299258': 428,
         'SUB14200269': 411,
         'SUB5118208': 402,
         'SUB13808474': 224,
         'ICSL_2016Q4': 166,
         'SUB12508014': 109,
         'SUB6300041': 101,
         'SUB11447241': 84,
         'SUB11035566': 60,
         'SUB12507860': 55,
         'GenomeConnect_hg19_81817': 53,
         'SUB12390657': 47,
         'SUB7304360': 41,
         'SUB14867261': 39,
    

No single submission name dominates as it did in 2024, in fact the largest group is those missing submission name entirely - maybe indicates this isn't as reliable an attribute as we thought.

In [21]:
def count_submitters(dataset):
    submitters = Counter()
    for cvs in dataset.iter_cvs():
        for scv in cvs.scvs:
            submitters[scv.submitter_id] += 1
    return submitters

In [22]:
previous_submitters = count_submitters(previous_set)

In [23]:
current_submitters = count_submitters(current_set)

In [26]:
prevention_id = '239772'

In [26]:
previous_submitters

Counter({'239772': 112072,
         '500031': 7003,
         '504895': 1652,
         '500034': 689,
         '506081': 287,
         '500026': 245,
         '506185': 230,
         '507119': 125,
         '1019': 113,
         '505999': 100,
         '1043': 84,
         '508034': 78,
         '508087': 54,
         '505993': 48,
         '500105': 48,
         '508359': 47,
         '504864': 45,
         '505998': 34,
         '1006': 26,
         '509068': 25,
         '507830': 23,
         '118451': 23,
         '506517': 21,
         '508506': 16,
         '249401': 15,
         '505265': 13,
         '507356': 12,
         '508927': 11,
         '506173': 11,
         '506437': 10,
         '506039': 10,
         '506663': 9,
         '506526': 7,
         '506672': 6,
         '500104': 5,
         '505952': 5,
         '506664': 5,
         '505219': 5,
         '506086': 5,
         '500062': 4,
         '508172': 4,
         '3': 4,
         '504828': 3,
         '165021': 

In [27]:
current_submitters

Counter({'239772': 144283,
         '500031': 7003,
         '504895': 1649,
         '500034': 689,
         '500026': 309,
         '506081': 287,
         '506185': 230,
         '1019': 135,
         '507119': 125,
         '505999': 119,
         '1043': 84,
         '508034': 77,
         '508087': 54,
         '505993': 48,
         '500105': 48,
         '508359': 46,
         '504864': 45,
         '505315': 39,
         '505998': 34,
         '1006': 26,
         '509068': 25,
         '507830': 23,
         '509541': 23,
         '118451': 22,
         '506517': 21,
         '249401': 16,
         '508506': 16,
         '505265': 13,
         '507356': 12,
         '508927': 11,
         '506173': 11,
         '506437': 10,
         '506039': 10,
         '506663': 9,
         '506672': 8,
         '506526': 7,
         '500104': 6,
         '509781': 6,
         '505952': 5,
         '507558': 5,
         '506664': 5,
         '505219': 5,
         '505991': 5,
         '50

The same submitter is by far the dominant one across this subset of the data in both 2024 and 2025.

In [4]:
previous_rcvs = {record.accession for record in previous_set}

In [5]:
current_rcvs = {record.accession for record in current_set}

In [40]:
len(previous_rcvs)

122432

In [41]:
len(current_rcvs)

154563

In [42]:
len(previous_rcvs - current_rcvs)

35

In [43]:
len(current_rcvs - previous_rcvs)

32166

In [23]:
paired_rcvs = []

# Find RCVs present in both sets, submitted by Prevention, initially part of the excluded submission and now not
for rcv in previous_rcvs & current_rcvs:
    good_rcv = False
    for previous_cvs in previous_set.iter_cvs():
        if previous_cvs.rcv.accession == rcv:
            good_rcv = (prevention_id in {scv.submitter_id for scv in previous_cvs.scvs}
                        and excluded_submission in {scv.submission_name for scv in previous_cvs.scvs})
            break
    if not good_rcv:
        continue
    for current_cvs in current_set.iter_cvs():
        if current_cvs.rcv.accession == rcv:
            good_rcv = good_rcv and (prevention_id in {scv.submitter_id for scv in current_cvs.scvs}
                                     and excluded_submission not in {scv.submission_name for scv in current_cvs.scvs})
            break
    if good_rcv:
        print("Found RCV to use:", rcv)
        paired_rcvs.append((previous_cvs, current_cvs))
        if len(paired_rcvs) >= 5:
            break

Found RCV to use: RCV003940464
Found RCV to use: RCV004532160
Found RCV to use: RCV003919663
Found RCV to use: RCV003955379
Found RCV to use: RCV004534523


In [24]:
pprint(previous_cvs.cvs_xml)

<ClinVarSet ID="207155321">
  <RecordStatus>current</RecordStatus>
  <Title>NM_021871.4(FGA):c.804G&gt;A (p.Glu268=) AND FGA-related disorder</Title>
  <ReferenceClinVarAssertion ID="9766709" DateLastUpdated="2024-06-29" DateCreated="2024-05-12">
    <ClinVarAccession Acc="RCV004534523" Version="1" Type="RCV" DateUpdated="2024-06-29" DateCreated="2024-05-12" />
    <RecordStatus>current</RecordStatus>
    <Classifications>
      <GermlineClassification>
        <ReviewStatus>criteria provided, single submitter</ReviewStatus>
        <Description DateLastEvaluated="2023-12-20" SubmissionCount="1">Likely benign</Description>
      </GermlineClassification>
    </Classifications>
    <Assertion Type="variation to disease" />
    <ObservedIn>
      <Sample>
        <Origin>germline</Origin>
        <Species TaxonomyId="9606">human</Species>
        <AffectedStatus>unknown</AffectedStatus>
      </Sample>
      <Method>
        <MethodType>clinical testing</MethodType>
      </Method>
     

In [25]:
pprint(current_cvs.cvs_xml)

<ClinVarSet ID="243249944">
  <RecordStatus>current</RecordStatus>
  <Title>NM_021871.4(FGA):c.804G&gt;A (p.Glu268=) AND FGA-related disorder</Title>
  <ReferenceClinVarAssertion ID="9766709" DateLastUpdated="2024-11-24" DateCreated="2024-05-12">
    <ClinVarAccession Acc="RCV004534523" Version="2" Type="RCV" DateUpdated="2024-11-24" DateCreated="2024-05-12" />
    <RecordStatus>current</RecordStatus>
    <Classifications>
      <GermlineClassification>
        <ReviewStatus>no assertion criteria provided</ReviewStatus>
        <Description DateLastEvaluated="2023-12-20" SubmissionCount="1">Likely benign</Description>
      </GermlineClassification>
    </Classifications>
    <Assertion Type="variation to disease" />
    <ObservedIn>
      <Sample>
        <Origin>germline</Origin>
        <Species TaxonomyId="9606">human</Species>
        <AffectedStatus>unknown</AffectedStatus>
      </Sample>
      <Method>
        <MethodType>clinical testing</MethodType>
      </Method>
      <Obs